# Preliminaries

In [19]:
import math
from sympy import symbols, sqrt, gcd, Integer, latex, floor, together, Add, factor, simplify, Matrix, eye, Symbol, root, resultant, expand, pprint
from IPython.display import display, Math
from fractions import Fraction


# uses together to recombine any sums of fractions and not break them apart.
def latex_fraction_compact(expr, parentheses_for_sum=True):
    expr = together(expr)  # combines into a single fraction
    num, den = expr.as_numer_denom()
    num_ltx = latex(num)
    if parentheses_for_sum and num.is_Add:
        num_ltx =  num_ltx 
    return r"\frac{" + num_ltx + '}{' + latex(den) + "}"

# Funzioni

## Matrix construction

In [3]:
def matrix_one_step(a):
    """ it returns the Möbius matrix for the step i-th """
    return Matrix([[0, 1],
                      [1, -a]])


In [4]:
def moebius_apply(M, x):
    """ Apply the Möbius transformation M to x """
    return (M[0,0]*x + M[0,1]) / (M[1,0]*x + M[1,1])
# M[0,0]=a, M[0,1]=1 etc


In [5]:

def build_matrix(period):
    # build the overall matrix for the period
    M = eye(2) # identity matrix of size 2x2
    
    for a in period:
        M = matrix_one_step(a)*M
    return M


## Frazioni Continuate

In [6]:
def Verify_fraction(gamma, period, preperiod):
    # Verify which of the two calculated roots is equal to the possibly periodic continued fraction represented by period and preperiod. returns True if equal, False otherwise.
    g = gamma

    if preperiod!=[]:
        for a in preperiod:
            if floor(g) != a:
                return False
            g = 1 / (g - floor(g))

    for a in period:
        if floor(g) != a:
            return False
        g = 1 / (g - floor(g))

    return True


In [7]:
def printa_fraction_continued(preperiod, period):

    # create the part of the period like "a, b, c"
    period_body = ", ".join(map(str, period))

    if preperiod == []:
        # no pre-period: [ \overline{period} ]
        cf_str = r"[" + r"\overline{" + period_body + r"}]"
        display(Math( r"\gamma=\alpha"+ r":= " + cf_str))
    else:
        pre_body = ", ".join(map(str, preperiod))
        cf_str = r"[" + pre_body + r"; " + r"\overline{" + period_body + r"}]"

        cf_str2= r"[" + r"\overline{" + period_body + r"}]"
        display(Math( r"\alpha"+ r":= " + cf_str))
        display(Math( r"\gamma"+ r":= " + cf_str2))


   


## Polinomio 

In [8]:
def root_poly(A,B,C):
    # Computes the roots of the polynomial Ax^2 +Bx+C 
    Delta= B**2 - 4*A*C

    gamma2= (-B-sqrt(Delta))/(2*A)

    gamma1= (-B+sqrt(Delta))/(2*A)

    return gamma1, gamma2
    

In [9]:
def polynomial_pure_periodic(period):
    """
    it construct the polynomial for gamma=[overline{period}], to do that we need to compute M such that x = M(x) where M is the transformation of the period.
    # The polynomial then became is M[1,0]x^2 + (M[1,1]-M[0,0])x - M[0,1] = 0
    """
    x = Symbol("x")
    M = build_matrix(period)

    A=M[0,0]
    B=M[0,1]
    C=M[1,0]
    D=M[1,1]
    T=D-A

    if gcd(C, gcd(T, B)) != 1:
        G = gcd(C, gcd(T, B))
        B = B // G     
        C = C // G
        T = T // G
    #Voglio un polinomio con coefficienti coprimi 


    gamma1, gamma2 = root_poly(C,T,-B)


    preperiod=[]

    verifica1 = Verify_fraction(gamma1, period, preperiod)
    verifica2 = Verify_fraction(gamma2, period, preperiod)

    # use Sympy's Integer to ensure they are treated as symbolic objects
    sym_A = Integer(C)
    sym_B = Integer(T)
    sym_delta = Integer(T**2 - 4*C*(-B))

 # We build the roots avoiding transforming them into sums of fractions.
  # Add(..., evaluate=False) prevents combinations that could distribute the denominator.
    num1 = Add(-sym_B, sqrt(sym_delta), evaluate=False)
    num2 = Add(sym_B, sqrt(sym_delta), evaluate=False)
    alpha1 = num1 / (2 * sym_A)
    alpha2 = num2 / (-2 * sym_A)


    if verifica1:
           

        display(Math(r"\text{The polynomial of } " + r"\gamma = " + latex_fraction_compact(alpha1)  + r" \text{ is: }" + str(C)  + r"x^2 + (" + str(T) + r") x + (" + str(-B) + r")"))
    
        

    if verifica2:
        display(Math(r"\text{The polynomial of } " + r"\gamma = " + latex_fraction_compact(alpha2)  + r" \text{ is: }" + str(C)  + r"x^2 + (" + str(T) + r") x + (" + str(-B) + r")"))

     


     
    
    return C,T,-B



## Convergenti


In [10]:
def Construct_convergentnth_from_continued_fraction(preperiod, period, n):
    """
    Construct the n-th convergent of the continued fraction with given preperiod and period.
    If n is less than the length of the preperiod, it constructs the convergent from the preperiod only.
    If n is greater than or equal to the length of the preperiod, it uses the period to extend the continued fraction.
    """
    cf_terms = preperiod[:]
    period_length = len(period)

 # Now compute the convergent
    if n<0:
        display(Math(r"\text{n must be non-negative}"))
        return 


    if n < len(preperiod):
        cf_terms = cf_terms[:n+1] # only preperiod terms needed
    
    else:

        cf_terms.extend(period * ((n - len(preperiod)+1) // period_length))

        cf_terms.extend(period[:(n - len(preperiod)+1) % period_length])



    
    numerator0 = Integer(cf_terms[0]) # P_{0}= a_0 Q_{0}=1
    denominator0 = Integer(1)

    if n==0:
       
        return numerator0 / denominator0    

    if n==1:
        return Integer(cf_terms[0]*cf_terms[1]+1) / Integer(cf_terms[1])
    
    
    numerator1 = Integer(cf_terms[0]*cf_terms[1]+1) # P_{1}= a_0a_1+1 Q_{1}=a_1
    denominator1 = Integer(cf_terms[1])
    

    for a in cf_terms[2:]:
        numerator0, numerator1 = numerator1, a * numerator1 + numerator0
        denominator0, denominator1 = denominator1, a * denominator1 + denominator0
    


    return numerator1 / denominator1

In [11]:
def Construct_convergentslist_from_continued_fraction(preperiod, period,n):
    """
    Construct the convergents until the nth of the continued fraction with given preperiod and period. so it gives n+1 numbers, since it includes the 0-th convergent.
    If n is less than the length of the preperiod, it constructs the convergent from the preperiod only.
    If n is greater than or equal to the length of the preperiod, it uses the period to extend the continued fraction.
    """
    
    cf_terms = preperiod[:]
    period_length = len(period)

 # Now compute the convergent
    if n<0:
        display(Math(r"\text{n must be non-negative}"))
        return 


    if n < len(preperiod):
        cf_terms = cf_terms[:n+1] # only preperiod terms needed
        
    
    else:
        need = n - len(preperiod) + 1
        full = need // period_length
        rest = need % period_length
        cf_terms.extend(period * full)
        cf_terms.extend(period[:rest])


    
    numerator0 = Integer(cf_terms[0]) # P_{0}= a_0 Q_{0}=1
    denominator0 = Integer(1)

    if n==0:
       
        return numerator0 / denominator0    

    if n==1:
        return (numerator0 / denominator0, Integer(cf_terms[0]*cf_terms[1]+1) / Integer(cf_terms[1]))
    
    
    numerator1 = Integer(cf_terms[0]*cf_terms[1]+1) # P_{1}= a_0a_1+1 Q_{1}=a_1
    denominator1 = Integer(cf_terms[1])
    
  

    num = [numerator0 / denominator0, numerator1 / denominator1]

   

    for a in cf_terms[2:]:
        numerator0, numerator1 = numerator1, a * numerator1 + numerator0
        denominator0, denominator1 = denominator1, a * denominator1 + denominator0
        num.append(numerator1 / denominator1)


    return num

In [ ]:
def from_convergentsstring_to_continued_fraction(convergents):
    """
    Given a list of convergents (as a string: so in particular it must be something of the type "1/2", "3/4", etc.), reconstruct the continued fraction representation.
    """
    
    num = []
    
    for frac in convergents:
        numerator = int(frac.split('/')[0])  # take first part (before '/')
        num.append(numerator)
    
    cf_terms = []
    cf_terms.append(num[0])
    cf_terms.append((num[1]-1)//num[0])
    for i in range(2, len(num)):
        a_i = (num[i] - num[i-2]) // num[i-1]
        cf_terms.append(a_i)

    
       
    # create the part of the period like "a, b, c"
    period_body = ", ".join(map(str, cf_terms))

    cf_str = r"[" + period_body + r"]"
    display(Math( r"\alpha"+ r":= " + cf_str))


    
    return 

In [23]:
def from_convergentsnumb_to_continued_fraction(convergents):
    """
    Given a list of convergents (as numbers): reconstruct the continued fraction representation.
    """
      
    
    num = []
    
    for frac in convergents:
        numerator = (Fraction(frac).limit_denominator()).numerator
        num.append(numerator)
    
    cf_terms = []
    cf_terms.append(num[0])
    cf_terms.append((num[1]-1)//num[0])
    for i in range(2, len(num)):
        a_i = (num[i] - num[i-2]) // num[i-1]
        cf_terms.append(a_i)

    
       
    # create the part of the period like "a, b, c"
    period_body = ", ".join(map(str, cf_terms))

    cf_str = r"[" + period_body + r"]"
    display(Math( r"\alpha"+ r":= " + cf_str))


    
    return 

# Codice intero


In [13]:
def fromcontinuedfractiontopolynomial(preperiod, period):
    """
    build the polynomial for alpha  = [preperiod, overline{period}]
    uses gamma and applies the cascade transformation of the preperiod.
    """
    x = Symbol("x")


    printa_fraction_continued(preperiod, period)
    
    
    (A,B,C)= polynomial_pure_periodic(period)

    if preperiod == []:
        return 


    N=build_matrix(preperiod)

    

   
    a= A*N[0,0]**2 + B*N[0,0]*N[1,0] + C*N[1,0]**2
    b=2*A*N[0,0]*N[0,1] + B*(N[0,0]*N[1,1] + N[0,1]*N[1,0]) + 2*C*N[1,0]*N[1,1]
    c=A*N[0,1]**2 + B*N[0,1]*N[1,1] + C*N[1,1]**2

    G = gcd(a, gcd(b, c))



    if G != 1:
        a = a // G     
        b = b // G
        c = c // G


    gamma1, gamma2 = root_poly(a,b,c)

    verifica1 = Verify_fraction(gamma1, period, preperiod)
    verifica2 = Verify_fraction(gamma2, period, preperiod)

    # We use Sympy's Integer to ensure they are treated as symbolic objects
    sym_A = Integer(a)
    sym_B = Integer(b)
    sym_delta = Integer(b**2 - 4*a*c)

 # We build the roots avoiding transforming them into sums of fractions.
  # Add(..., evaluate=False) prevents combinations that could distribute the denominator.

    num1 = Add(-sym_B, sqrt(sym_delta), evaluate=False)
    num2 = Add(sym_B, sqrt(sym_delta), evaluate=False)
    alpha1 = num1 / (2 * sym_A)
    alpha2 = num2 / (-2 * sym_A)


    if verifica1:
        display(Math(r"\text{The polynomial of } " + r"\alpha = " + latex_fraction_compact(alpha1)  + r" \text{ is: }" + str(a)  + r"x^2 + (" + str(b) + r") x + (" + str(c) + r")"))
        
    
        

    if verifica2:
        display(Math(r"\text{The polynomial of } " + r"\alpha = " + latex_fraction_compact(alpha2)  + r" \text{ is: }" + str(a)  + r"x^2 + (" + str(b) + r") x + (" + str(c) + r")"))

     


    return






In [14]:
fromcontinuedfractiontopolynomial(([1,5,7]), [3,4])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [15]:
Construct_convergentslist_from_continued_fraction([1,5,7], [3,4],8)

[1,
 6/5,
 43/36,
 135/113,
 583/488,
 1884/1577,
 8119/6796,
 26241/21965,
 113083/94656]

In [18]:
Construct_convergentnth_from_continued_fraction([1,5,7], [3,4],8)

113083/94656

In [ ]:
from_convergentsstring_to_continued_fraction(['1/1', '6/5', '43/36', '135/113', '583/488', '1884/1577'])

<IPython.core.display.Math object>

In [24]:
from_convergentsnumb_to_continued_fraction([1,
 6/5,
 43/36,
 135/113,
 583/488,
 1884/1577,
 8119/6796,
 26241/21965,
 113083/94656])

<IPython.core.display.Math object>